# Zugriffsdaten-Analyse

## Datenimport

### Parsing

In [3]:
import apachelogs

parser = apachelogs.LogParser('{} "%{{Unknown}}i"'.format(apachelogs.COMBINED))

#### Beispiel

In [4]:
data = parser.parse('207.46.13.115 - - [22/Jan/2019:03:56:29 +0330] "GET /image/46131/productModel/100x100 HTTP/1.1" 200 1981 "-" "Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)" "-"\n').directives

print("Host: {}".format(data["%h"]))
print("Logname: {}".format(data["%l"]))
print("User: {}".format(data["%u"]))
print("Timestamp: {}".format(data["%t"]))
print("Request line: {}".format(data["%r"]))
print("Status: {}".format(data["%>s"]))
print("Bytes sent: {}".format(data["%b"]))
print("Referer: {}".format(data["%{Referer}i"]))
print("User-Agent: {}".format(data["%{User-Agent}i"]))

Host: 207.46.13.115
Logname: None
User: None
Timestamp: 2019-01-22 03:56:29+03:30
Request line: GET /image/46131/productModel/100x100 HTTP/1.1
Status: 200
Bytes sent: 1981
Referer: None
User-Agent: Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)


### Einlesen der Datei

In [5]:
from pyspark.sql import Row
import pandas as pd

file = open('data/access.log')
rows = []
for line in file.readlines()[:100]:
    entry = parser.parse(line).directives
    rows.append({
        "host": entry["%h"],
        "logname": entry["%l"],
        "user": entry["%u"],
        "timestamp": entry["%t"],
        "request_line": entry["%r"],
        "status": entry["%>s"],
        "bytes_sent": entry["%b"],
        "referer": entry["%{Referer}i"],
        "user_agent": entry["%{User-Agent}i"],
    })
file.close()

pd_df = pd.DataFrame(rows, columns=rows[0].keys())
pd_df.head()

,host,logname,user,timestamp,request_line,status,bytes_sent,referer,user_agent
0,54.36.149.41,None,None,2019-01-22 03:56:14+03:30,GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%D...,200,30577,None,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...
1,31.56.96.51,None,None,2019-01-22 03:56:16+03:30,GET /image/60844/productModel/200x200 HTTP/1.1,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
2,31.56.96.51,None,None,2019-01-22 03:56:16+03:30,GET /image/61474/productModel/200x200 HTTP/1.1,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
3,40.77.167.129,None,None,2019-01-22 03:56:17+03:30,GET /image/14925/productModel/100x100 HTTP/1.1,200,1696,None,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
4,91.99.72.15,None,None,2019-01-22 03:56:17+03:30,GET /product/31893/62100/%D8%B3%D8%B4%D9%88%D8...,200,41483,None,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...


In [6]:
pd_df.to_csv("data/access.log.csv", index=False)

## Spark

### Setup

In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.appName("main").master("spark://localhost:7077").getOrCreate()

22/06/19 22:02:19 WARN Utils: Your hostname, Pauls-Mac-mini-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.25 instead (on interface en0)
22/06/19 22:02:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/19 22:02:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv("data/access.log.csv")

22/06/19 22:02:35 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (192.168.80.2 executor 0): java.io.StreamCorruptedException: invalid stream header: 73720025
	at java.io.ObjectInputStream.readStreamHeader(ObjectInputStream.java:938)
	at java.io.ObjectInputStream.<init>(ObjectInputStream.java:396)
	at org.apache.spark.serializer.JavaDeserializationStream$$anon$1.<init>(JavaSerializer.scala:64)
	at org.apache.spark.serializer.JavaDeserializationStream.<init>(JavaSerializer.scala:64)
	at org.apache.spark.serializer.JavaSerializerInstance.deserializeStream(JavaSerializer.scala:127)
	at org.apache.spark.serializer.JavaSerializerInstance.deserialize(JavaSerializer.scala:114)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:466)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)

22/06/19 22:02:35 ERROR Task

Py4JJavaError: An error occurred while calling o26.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (192.168.80.2 executor 0): java.io.StreamCorruptedException: invalid stream header: 73720025
	at java.io.ObjectInputStream.readStreamHeader(ObjectInputStream.java:938)
	at java.io.ObjectInputStream.<init>(ObjectInputStream.java:396)
	at org.apache.spark.serializer.JavaDeserializationStream$$anon$1.<init>(JavaSerializer.scala:64)
	at org.apache.spark.serializer.JavaDeserializationStream.<init>(JavaSerializer.scala:64)
	at org.apache.spark.serializer.JavaSerializerInstance.deserializeStream(JavaSerializer.scala:127)
	at org.apache.spark.serializer.JavaSerializerInstance.deserialize(JavaSerializer.scala:114)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:466)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.uti

In [ ]:
df.count()